<a href="https://colab.research.google.com/github/MengOonLee/LLM/blob/main/References/LangChain/ipynb/T10_Local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash

pip install --no-cache-dir -qU \
    torch torchvision lightning  transformers accelerate \
    langchain langgraph langchain-core langchain-community \
    langchain-huggingface

In [3]:
import os
import warnings
warnings.filterwarnings("ignore")
from langchain_huggingface import llms
# import transformers
import time

model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
# model_id = "deepseek-ai/DeepSeek-V3.2"
hf = llms.HuggingFacePipeline.from_model_id(
    model_id=model_id,
    task="text-generation"
)

# model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
# local_path = "./models"
# os.makedirs(local_path, exist_ok=True)

# start_time = time.time()
# model = transformers.AutoModelForCausalLM.from_pretrained(
#     pretrained_model_name_or_path=model_name)
# model.save_pretrained(save_directory=local_path)
# tokenizer = transformers.AutoTokenizer.from_pretrained(
#     pretrained_model_name_or_path=model_name)
# tokenizer.save_pretrained(save_directory=local_path)
# end_time = time.time() - start_time

# print("Time taken: %.2f"%(end_time))

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 6a39cb5d-267a-4dd1-b1ea-39e78fa69c11)')' thrown while requesting HEAD https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/resolve/main/config.json
Retrying in 1s [Retry 1/5].
Device set to use cpu


In [4]:
from langchain_core.prompts import PromptTemplate
import time

template = """
Question: {question}
"""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf.bind(skip_prompt=True)

question = "What is SWIFT Financial messaging?"

start_time = time.time()
response = chain.invoke({"question": question})
end_time = time.time() - start_time

print("Time take: %.4f, %s"%(end_time, response))

Time take: 175.8994, The user is in the US, and wants to send a message to a US-based bank via SWIFT.
The user is not familiar with SWIFT, so the question should be explained in simple terms.

The user is also in the US, so the message should be in US dollars. They want to make sure they get the correct amount.
The user is looking for a SWIFT financial messaging service that is easy to use.
I should include the SWIFT service code for each of the banks I choose, but not necessary if it's the same banks. Probably only for the banks I select.

How to explain the SWIFT in simple terms?

Possible SWIFT banks I can think of are JPMorgan, Capital One, Wells Fargo, and Citi. But I might be wrong. Maybe not all of these are US banks, but I think they are.
Alright, so the user wants to send a message to a US-based bank via SWIFT. They're not familiar with SWIFT, so I need to explain it in simple terms. Also, I need to provide the SWIFT code for each of the banks they might use, if they choose th

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=local_path)
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=local_path)

hf_pipeline = pipeline(model=model, tokenizer=tokenizer,
    task="text-generation")
llm = HuggingFacePipeline(pipeline=hf_pipeline)

In [ ]:
from langchain_core.prompts import PromptTemplate
import time

template = """
Question: {question}
"""
prompt = PromptTemplate.from_template(template)

chain = prompt | llm

question = "What is SWIFT messaging?"

start_time = time.time()
response = chain.invoke({"question": question})
end_time = time.time() - start_time

print("Time take: %.4f, %s"%(end_time, response))